<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="center" width="50%">

# Xarray 3: Datasets and Plotting
---

## Overview
1. The Xarray `Dataset`
1. Open and select a variable from the ERA-5 dataset
1. Convert geopotential to height
1. Create a contour plot

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| Xarray Lessons 1-2| Necessary | |

* **Time to learn**: 15 minutes

## Imports

In [ ]:
import xarray as xr
import numpy as np
from metpy.units import units
import metpy.calc as mpcalc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Work with an Xarray `Dataset`

An Xarray [Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html) consists of one or more `DataArray`s. Let's open the same NetCDF file as in the first Xarray notebook, but as a `Dataset`.

In [ ]:
ds = xr.open_dataset('/spare11/atm533/data/2012103000_z500_era5.nc')

Let's see what this Dataset looks like.

In [ ]:
ds

It's represented somewhat similarly to a DataArray, but now we have support for multiple data variables. In this case,
1. There is just one data variable: z (Geopotential)
2. The coordinate variables are longitude, latitude, and time
3. The data variable has three dimensions (time x latitude x longitude). 
4. Although time is a dimension, there is only one time value on that coordinate.
5. The Dataset itself has attributes. The attributes of the coordinate and data variables are separately accessible.

#### Select a data variable from the Dataset.</span>

In [ ]:
Z = ds['z'] # Similar to selecting a Series in Pandas

We can programmatically retrieve various attributes of the data variable that we browsed interactively above.

In [ ]:
Z

In [ ]:
Z.shape

In [ ]:
Z.dims

We see that Z is a 3-dimensional DataArray, with time as the first dimension.

In [ ]:
Z.units

#### Convert geopotential to height

#### Print out the representation of our data variable. We can see it's a `DataArray`.

In [ ]:
print (Z) # Text-based, as opposed to HTML-prettified output

The units are already part of the data variable. We can use `MetPy`'s `units` and `calc` libraries to  [convert geopotential to height](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.geopotential_to_height.html). We'll assign a new object to the output of the function.

In [ ]:
HGHT = mpcalc.geopotential_to_height(Z)

#### Print out its representation. Note that the array values have changed, and there is a unit attached to the `DataArray`.

In [ ]:
print (HGHT)

<div class="alert alert-warning"><b>Note: </b>Why are we using `print`  to show what the HGHT object looks like? If we didn't, the array's values would be rendered in an HTML-friendly, aka *pretty-printed* manner ... while it looks nice, it can take frightfully long to render if the array is large.
</div>

Now we can make a map with contours of 500 hPa height!

#### Create a contour plot of gridded data

We will use Cartopy and Matplotlib to plot our 500 hPa height contours. Matplotlib has two contour methods:
1. Line contours: ax.contour
1. Filled contours: ax.contourf

#### We will draw contour lines in meters, with a contour interval of 60 m.

#### As we've done before, let's first define some variables relevant to Cartopy.

In [ ]:
cLon = -80
cLat = 35
lonW = -100
lonE = -60
latS = 20
latN = 50
proj_map = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)
proj_data = ccrs.PlateCarree() # Our data is lat-lon; thus its native projection is Plate Carree.
res = '50m'

Now define the range of our contour values and a contour interval. 60 m is standard for 500 hPa.

In [ ]:
minVal = 4680
maxVal = 6060
cint = 60
cintervals = np.arange(minVal, maxVal, cint)
cintervals

Matplotlib's contour methods require three arrays to be passed to them ... x- and y- arrays (longitude and latitude in our case), and a 2-d array (corresponding to x- and y-) of our data variable. So we need to extract the latitude and longitude coordinate variables from our DataArray. We'll also extract the third coordinate value, time.

In [ ]:
lats = ds.latitude 
lons = ds.longitude
times = ds.time

We're set to make our map. After creating our figure, setting the bounds of our map domain, and adding cartographic features, we will plot the contours. We'll assign the output of the contour method to an object, which will then be used to label the contour lines.

In [ ]:
fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=proj_map)
ax.set_extent ([lonW,lonE,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))
CL = ax.contour(lons,lats,HGHT,cintervals,transform=proj_data,linewidths=1.25,colors='green')
ax.clabel(CL, inline_spacing=0.2, fontsize=11, fmt='%.0f');

<div class="alert alert-danger"><h3>Didn't work! The end of the traceback reads:</h3>
    <code>TypeError: Input z must be 2D, not 3D</code>
</div>

<div class="alert alert-info">
<h4>In other words, our geopotential height array is not two dimensional in x and y ... recall that it also has *time* as its first dimension! Even though there is only one time value in this dimension, we need to eliminate this dimension from Z.</h4>
</div>

We'll learn other ways to abstract out dimensions other than x and y (remember, there could be additional dimensions in our Datasets or DataArrays, such as vertical levels or ensemble members) in later notebooks, but for now, we can simply select the first (and only) element from the time dimension.

In [ ]:
HGHT = HGHT[0,:,:] # Select the first element of the leading (time) dimension, and all lons and lats.

In [ ]:
HGHT

Now try again.

In [ ]:
fig = plt.figure(figsize=(18,12))
ax = plt.subplot(1,1,1,projection=proj_map)
ax.set_extent ([lonW,lonE,latS,latN])
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))
CL = ax.contour(lons,lats,HGHT,cintervals,transform=proj_data,linewidths=1.25,colors='green')
ax.clabel(CL, inline_spacing=0.2, fontsize=11, fmt='%.0f');

#### There we have it! There are definitely some things we can improve about this plot (such as the lack of an informative title, and the fact that our contour labels seem not to be appearing on every contour line), but we'll get to that soon!

---
## Summary
* An Xarray `Dataset` consists of one or more `DataArrays`.
* MetPy can perform unit-aware calculations and conversions on Xarray objects.
* Since Xarray `DataArray`s often have more than x- and y- dimensions, care must be taken to reduce dimensionality for the purposes of Matplotlib functions such as `contour` and `contourf`.

### What's Next?
In the next notebook, we'll work with multiple `Dataset`s.

## Resources and References
1. [The Xarray Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html)
1. [Units in MetPy](https://unidata.github.io/MetPy/latest/tutorials/unit_tutorial.html#sphx-glr-tutorials-unit-tutorial-py)
1. [MetPy Calculations](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.html#module-metpy.calc)
1. [Matplotlib Contour Line Plots](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.contour.html)
1. [Matplotlib Contour Fill Plots](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.contourf.html)